<a href="https://colab.research.google.com/github/ankersp/thesis/blob/main/Sentence%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentence extraction


## Import libraries


In [ ]:
# Install required libraries
!pip install spacy
!pip install tika

# Import necessary libraries
import spacy  # For sentence extraction
from tika import parser  # For report extraction

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32624 sha256=b3f04d68ba55a737c35bfdf2bcc22234bd5478be303fb52426e6ffb388520ec1
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


## Step 1: Load models

I'm using the models of the [ESG BERT project](https://huggingface.co/ESGBERT).



# Step 2: loading data ##

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

# List the contents of your Drive directory
os.listdir('/content/drive/My Drive/KU/kandidat/speciale/R')


['ESG.csv',
 'danish_texts.csv',
 'translated_texts.csv',
 'unique_data.csv',
 'Rplot.png',
 'metadata.csv',
 'data_eng.csv',
 'data_final.csv',
 'classification_results.csv',
 'train_classified.csv',
 'test_classification_results.csv',
 'test_classified.csv',
 'train_data.csv',
 'test_data.csv',
 'test_adjusted_data.csv',
 'train_adjusted_data.csv',
 'eikon_ESG_scores_with_predictions.csv',
 'train_CSR_data_with_predicted_ESG_scores.csv',
 'train_with_actions.csv',
 'train_classified_with_sentiment.csv',
 'train_with_actions_new.csv',
 'train_grouped_with_actions.csv',
 'test_classified_with_sentiment_final.csv',
 'test_with_actions.csv',
 'test_grouped_with_actions.csv',
 'word2vec-model-2024-11-11-12-16-52.bin',
 'consecutive_years_data.csv',
 'panel_data_simil.csv',
 'produktionsenheder.csv',
 'tilsynsfrekvenser.csv',
 'Erhvervspuljen 2020-2024.xlsx',
 'subs.R',
 'acf.R',
 'Event_Study_Exercise.R',
 'Event_Study_Data.csv',
 'Event study.R',
 'Metadata cleaning and qta stuff.R',
 'd

In [ ]:
import pandas as pd

# Specify the path to the file in Google Drive
file_path = '/content/drive/My Drive/KU/kandidat/speciale/R/data_final_clean.csv'
data = pd.read_csv(file_path)


In [ ]:
# Install spacy and the language model
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 122.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pandas as pd

# Load the spacy model for English
nlp = spacy.load('en_core_web_sm')

# Function to process text into sentences
def text_to_sentences(text):
    # Use spacy to split the text into sentences
    about_doc = nlp(text[:1000000])

    # Extract the sentences
    sequences = list(map(str, about_doc.sents))

    # Remove new line characters and filter out empty sentences
    sentences = [x.replace("\n", "") for x in sequences if x != ""]

    # Only keep sentences that start with an uppercase letter
    sentences = [x for x in sentences if x[0].isupper()]

    return sentences

# Apply the function to the 'final_text' column in your data
data['sentences'] = data['final_text'].apply(text_to_sentences)

# Check the result for the first few rows
print(data[['doc_id', 'sentences']].tail())


          doc_id                                          sentences
11452  doc_12067  [Responsible supply chain management BEUMER Gr...
11453  doc_12068  [Corporate Social Responsibility, StatementThe...
11454  doc_12069  [Report on Corporate Social Responsibility, cf...
11455  doc_12070  [The FairWind group’s CSR policy and responsib...
11456  doc_12071  [Going concern Financing The Group’s aircraft ...


In [ ]:
# List all column names
print("Column names:", long_data.columns.tolist())


Column names: ['Unnamed: 0', 'doc_id', 'CVR', 'Year', 'Revenue', 'Result', 'Corporate.form', 'Region', 'Industry', 'Class', 'final_text', 'Industry_new', 'Corporate_structure', 'Class_new', 'sentences']


In [ ]:
# Ensure 'sentences' column exists
if 'sentences' in data.columns:
    # Explode the dataframe to make each sentence a row
    long_data = data.explode('sentences', ignore_index=True)

    # Keep only the required columns
    columns_to_keep = ['doc_id', 'CVR', 'Year', 'sentences']
    long_data = long_data[columns_to_keep]

    # Add a sequential ID column
    long_data['id'] = range(1, len(long_data) + 1)

    # Save the dataframe to a standard CSV
    save_path = '/content/drive/My Drive/KU/kandidat/speciale/R/sentences_long_format.csv'
    long_data.to_csv(save_path, index=False)

    print(f"Transformed dataframe saved to: {save_path}")
else:
    print("Column 'sentences' not found in the dataframe.")


Transformed dataframe saved to: /content/drive/My Drive/KU/kandidat/speciale/R/sentences_long_format.csv
